**Journal prediction on APS citations with Node2Vec**



Goal is to predict the journal of 2013 papers using graph embeddings learned from the citation network.I will build Node2Vec embeddings on the full unlabeled graph, then train simple classifiers on 2010 to 2012 and evaluate on 2013. Node prediction problem

In [ ]:
import networkx as nx
from node2vec import Node2Vec
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings("ignore")
import numpy as np, pickle, warnings
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.ensemble import RandomForestClassifier # type: ignore
from sklearn.svm import LinearSVC # type: ignore
from sklearn.model_selection import GridSearchCV, PredefinedSplit # type: ignore
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, top_k_accuracy_score # type: ignore
np.random.seed(42)

Read the Pajek network file with doi and year info

In [16]:
def read_pajek_citations(filename):
    G = nx.Graph()
    
    with open(filename, 'r') as file:
        # Read header
        first_line = file.readline()
        n_nodes = int(first_line.split()[1])
        
        # Read nodes
        for line in tqdm(file, total=n_nodes, desc="Nodes"):
            if line.startswith("*"):
                break
            else:
                parts = line.split("\"")
                node_id = parts[0].strip()
                doi = parts[1].strip()
                year = int(parts[2].strip())
                
                # Extract journal from DOI: "10.1203/PhysRevA.81.012102" -> "PhysRevA")
                journal = doi.split("/")[1].split(".")[0] if "/" in doi else "Unknown"
                
                G.add_node(node_id, doi=doi, year=year, journal=journal)
        
        # Read edges
        edges = []
        for line in tqdm(file, desc="Edges"):
            edge = line.split()[:2]
            if len(edge) == 2:
                edges.append(tuple(edge))
        
        G.add_edges_from(edges)
    
    return G

G = read_pajek_citations("data/aps_citations.net")


Nodes: 100%|██████████| 56473/56473 [00:00<00:00, 474752.48it/s]
Edges: 200592it [00:00, 1835820.81it/s]


200592 represent the lines parsed not edges

Lets verify network statistics

In [6]:
n_nodes = G.number_of_nodes()
n_edges = G.number_of_edges()

print(f"Nodes: {n_nodes:,}")
print(f"Edges: {n_edges:,}")

Nodes: 56,473
Edges: 200,353


Assertions

In [8]:
assert n_nodes == 56473, f"Expected 56,473 nodes but got {n_nodes:,}"
assert n_edges == 200353, f"Expected 200,353 edges but got {n_edges:,}"

Year distribution

In [11]:
year_counts = Counter(G.nodes[n]['year'] for n in G.nodes())
for year in sorted(year_counts.keys()):
    print(f" {year}: {year_counts[year]:,} papers")

 2010: 12,680 papers
 2011: 13,862 papers
 2012: 15,268 papers
 2013: 14,663 papers


The dataset almost balanced across years with a gradual increase in the number of papers from 2010 to 2012 and a slight drop in 2013.
This means that the if we do temporal split (training on 2010–2012 and testing on 2013) it will provides a reasonable and representative division for model evaluation.

Journal distribution

In [21]:
journal_counts = Counter(G.nodes[n]['journal'] for n in G.nodes())
print(f"\nNumber of unique journals: {len(journal_counts)}")
for journal, count in journal_counts.items():
    print(f"  {journal}: {count:,} papers")


Number of unique journals: 9
  PhysRevA: 8,310 papers
  PhysRevB: 16,060 papers
  PhysRevC: 3,157 papers
  PhysRevD: 10,461 papers
  PhysRevE: 5,653 papers
  PhysRevLett: 12,116 papers
  PhysRevSTAB: 348 papers
  PhysRevX: 179 papers
  RevModPhys: 189 papers


Train test split


In [22]:
train_nodes = [n for n in G.nodes() if G.nodes[n]['year'] <= 2012]
test_nodes = [n for n in G.nodes() if G.nodes[n]['year'] == 2013]
print(f"\nTrain set (2010-2012): {len(train_nodes):,} papers")
print(f"Test set (2013): {len(test_nodes):,} papers")


Train set (2010-2012): 41,810 papers
Test set (2013): 14,663 papers


Lets now generate Node2Vec embedding. I will keep higher dimensions to capture more info. I will keep p=q=1 (mix BFS abd DFS) because it works well for citation networks. I will also keep more walks and longer walks to improve embedding quality. 

In [ ]:
d = 128  # embedding dimension (higher for large network)
r = 30   # number of walks per node
l = 80   # walk length
p = 1.0  # return parameter (balanced)
q = 1.0  # in-out parameter (balanced)
window = 10  # context window for Word2Vec
min_count = 1  # minimum word count
workers = 8  # parallel workers

node2vec = Node2Vec(
    G, 
    dimensions=d, 
    walk_length=l, 
    num_walks=r, 
    p=p, 
    q=q,
    workers=workers,
    quiet=False  # Show progress
)

model = node2vec.fit(window=window, min_count=min_count, batch_words=4)
wv = model.wv


Save embeddings and metadata and test train split

In [ ]:
embeddings = {node: wv[node] for node in G.nodes()}
with open('data/aps_embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)


metadata = {
    node: {
        'journal': G.nodes[node]['journal'],
        'year': G.nodes[node]['year'],
        'doi': G.nodes[node]['doi']
    }
    for node in G.nodes()
}
with open('data/aps_metadata.pkl', 'wb') as f:
    pickle.dump(metadata, f)


split_info = {
    'train_nodes': train_nodes,
    'test_nodes': test_nodes
}
with open('data/aps_train_test_split.pkl', 'wb') as f:
    pickle.dump(split_info, f)

In [23]:
print(f"Training papers (2010-2012): {len(train_nodes):,}")
print(f"Test papers (2013): {len(test_nodes):,}")

Training papers (2010-2012): 41,810
Test papers (2013): 14,663


Lets again reload saved embeddings and metadata and test train split

In [26]:


with open('data/aps_embeddings.pkl', 'rb') as f:
    emb = pickle.load(f)
with open('data/aps_metadata.pkl', 'rb') as f:
    meta = pickle.load(f)
with open('data/aps_train_test_split.pkl', 'rb') as f:
    split = pickle.load(f)

train_nodes = split['train_nodes']
test_nodes  = split['test_nodes']

print(f"  Train nodes: {len(train_nodes):,}")
print(f"  Test nodes: {len(test_nodes):,}")

  Train nodes: 41,810
  Test nodes: 14,663


All good so far. I loaded the saved embeddings and metadata and asserted. I will train on 2010 to 2012 and test on 2013. For a light and defensible tuning step, I will select the Logistic Regression regularization C by validating on 2012 and training on 2010 to 2011. I will also train Random Forest and Linear SVM as baselines.Then I will report accuracy, macro F1 and for Logistic Regression also top 3 accuracy.I will also show a confusion matrix with a consistent label order.

Pack features and labels

In [27]:
def pack(nodes):
    X, y, yrs = [], [], []
    for n in nodes:
        if n in emb and n in meta:
            X.append(emb[n])
            y.append(meta[n]['journal'])
            yrs.append(meta[n]['year'])
    return np.vstack(X), np.array(y), np.array(yrs)

X_train_full, y_train_full, years_train_full = pack(train_nodes)  # 2010–2012
X_test,        y_test,        years_test     = pack(test_nodes)   # 2013

print("Train:", X_train_full.shape, "Test:", X_test.shape)
print("Train journals:", len(set(y_train_full)), "Test journals:", len(set(y_test)))

Train: (41810, 128) Test: (14663, 128)
Train journals: 9 Test journals: 9


Train and test sets have equal journal coverage and consistent 128-Dim embeddings, ensuring balanced representation and structural continuity.

In [29]:
# Sanity Check of Label Coverage
seen = set(y_train_full)
mask_seen = np.isin(y_test, list(seen))
print(f"Seen-label coverage in 2013: {mask_seen.mean():.3f}")
X_test_seen, y_test_seen = X_test[mask_seen], y_test[mask_seen]

Seen-label coverage in 2013: 1.000


All 2013 papers belong to journals already present in the training set so seen-label coverage is 1. This will make sure that evaluation will be fair and the model is not tested on unseen classes.

Lets now do temporal hyperparameter tuning for logistic regression using 2012

In [33]:
is_val = (years_train_full == 2012)
ps = PredefinedSplit(test_fold=np.where(is_val, 0, -1))
lr_base = LogisticRegression(max_iter=1000, multi_class="multinomial",
                             class_weight="balanced", solver="lbfgs", random_state=42)
grid = GridSearchCV(lr_base, {"C": [0.5, 1.0, 2.0]},
                    scoring="f1_macro", cv=ps, n_jobs=-1, refit=True)
grid.fit(X_train_full, y_train_full)
clf_lr = grid.best_estimator_
print("Best Logistic Regression params:", grid.best_params_)

Best Logistic Regression params: {'C': 0.5}


Smaller C means stronger regularization. This shows that (C = 0.5) a slightly stronger penalty on large weights gives better generalization for this dataset.

Lets build RF and SVM models with no heavy hyperparameter tuning to keep it light

In [34]:
clf_rf  = RandomForestClassifier(n_estimators=300, class_weight="balanced_subsample",
                                 random_state=42, n_jobs=-1).fit(X_train_full, y_train_full)
clf_svm = LinearSVC(max_iter=2000, random_state=42).fit(X_train_full, y_train_full)

Evaluation helper function

In [35]:
def eval_model(name, clf, X, y, topk=False):
    yhat = clf.predict(X)
    acc = accuracy_score(y, yhat)
    f1m = f1_score(y, yhat, average="macro", zero_division=0)
    out = {"acc": acc, "f1_macro": f1m, "yhat": yhat}
    if topk and hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X)
        out["top3"] = top_k_accuracy_score(y, proba, k=3)
    print(f"{name:18s}  Acc={acc:.4f}  MacroF1={f1m:.4f}" + (f"  Top3={out['top3']:.4f}" if "top3" in out else ""))
    return out


Evaluations

In [39]:
res_lr_seen  = eval_model("LogisticRegression", clf_lr, X_test_seen, y_test_seen, topk=True)
res_rf_seen  = eval_model("RandomForest",       clf_rf, X_test_seen, y_test_seen)
res_svm_seen = eval_model("LinearSVM",          clf_svm, X_test_seen, y_test_seen)

LogisticRegression  Acc=0.5797  MacroF1=0.4860  Top3=0.9168
RandomForest        Acc=0.7166  MacroF1=0.5514
LinearSVM           Acc=0.6988  MacroF1=0.5213


top-3 = 0.9168 shows that if the LR classifier can name three possible journals for each paper, the correct journal is in that list about 92% of the time.

Lets now choose best by Macro-F1 on seen test

In [42]:
candidates = {"LR": res_lr_seen, "RF": res_rf_seen, "SVM": res_svm_seen}
best_name = max(candidates, key=lambda k: candidates[k]["f1_macro"])
best_pred = candidates[best_name]["yhat"]
print(f"\nBest model on seen test: {best_name}")


Best model on seen test: RF


Insights: RF is best at first-try correctness (about 72% accuracy, macro-F1 ≈ 0.55). While LR is not as good on the very first guess but it is great at shortlisting the right journal (top-3 ≈ 92%). Linear SVM is close to Random Forest but a bit behind.

Classification Report

In [43]:
print(classification_report(y_test_seen, best_pred, zero_division=0))

              precision    recall  f1-score   support

    PhysRevA       0.71      0.79      0.75      2264
    PhysRevB       0.71      0.92      0.80      3864
    PhysRevC       0.83      0.83      0.83       845
    PhysRevD       0.83      0.96      0.89      2761
    PhysRevE       0.76      0.66      0.71      1589
 PhysRevLett       0.42      0.22      0.29      3124
 PhysRevSTAB       0.70      0.69      0.69        90
    PhysRevX       0.00      0.00      0.00        84
  RevModPhys       0.00      0.00      0.00        42

    accuracy                           0.72     14663
   macro avg       0.55      0.56      0.55     14663
weighted avg       0.68      0.72      0.68     14663



Commentary: Test model on the 2013 test set was Random Forest with accuracy around 0.72 and macro F1 around 0.55. Logistic Regression produced a strong top 3 accuracy near 0.92 which shows that the correct journal is often among the top three suggestions even when the top one is incorrect. Performance is strong for PhysRevB and PhysRevD and reasonable for PhysRevA and PhysRevC. PhysRevLett and PhysRevX remain challenging due to class imbalance and overlap in citation neighborhoods.

Lets save final resuts

In [46]:
final = {
"best_model_seen": best_name,
"lr_params": grid.best_params_,
"seen_coverage": float(mask_seen.mean()),
"seen_metrics": {k: {"acc": v["acc"], "f1_macro": v["f1_macro"], **({"top3": v["top3"]} if "top3" in v else {})}
for k, v in candidates.items()}
}
with open("data/aps_classification_results.pkl", "wb") as f:
    pickle.dump(final, f)
